# Project 2: Housing Price Prediction


The project should be done using Regression prediction techniques ( with feature scaling and regularization ) as covered in class.
The goal is to minimize the RMS ***percentage*** error ( root mean squared percentage error - RMSPE ) on your prediction of the house sales price.



Any of the available features can be used in the regression, but a minimum set of variables that do the job should be the ultimate goal.



Be sure to go through the whole data science process and document as such in your Jupyter notebook.



This project will have less direct "To Do" guidance and the progression through the data science process will be more individualized this time around.
We will talk about all the issues during class so you're not going to be out on a ledge with this one, I just want to move you towards performing a data science project on your own eventually.



A data dictionary file is available at AWS S3 at [Housing Data Dictionary]( https://ddc-datascience.s3.amazonaws.com/Projects/Project.2-Housing/Housing%20-%20Data%20Documentation.pdf ).

The data is available on AWS S3 at https://ddc-datascience.s3.amazonaws.com/Projects/Project.2-Housing/Data/Housing.Data.csv .


In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import statsmodels as sm
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
import statsmodels.api as sm

In [34]:
df = pd.read_csv('/Users/joeyrobak/Downloads/Housing.Data - Housing.Data.csv')

# Here I'm just importing our data, The data is available on AWS S3 at https://ddc-datascience.s3.amazonaws.com/Projects/Project.2-Housing/Data/Housing.Data.csv .


In [35]:
df.head().transpose()

#path of least resistence, drop everything but the floats first

,0,1,2,3,4
PID,905101070,905101330,903454090,533244030,909252020
MS SubClass,20,90,50,60,70
MS Zoning,RL,RL,RM,FV,RL
Lot Frontage,62.0,72.0,50.0,68.0,60.0
Lot Area,14299,10791,9000,7379,7200
...,...,...,...,...,...
Mo Sold,7,10,12,4,4
Yr Sold,2007,2006,2007,2010,2009
Sale Type,WD,WD,WD,WD,WD
Sale Condition,Normal,Normal,Normal,Normal,Normal


In [36]:
df.info()
# 400 columns after hot encoding ()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2637 entries, 0 to 2636
Data columns (total 81 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   PID              2637 non-null   int64  
 1   MS SubClass      2637 non-null   int64  
 2   MS Zoning        2637 non-null   object 
 3   Lot Frontage     2188 non-null   float64
 4   Lot Area         2637 non-null   int64  
 5   Street           2637 non-null   object 
 6   Alley            180 non-null    object 
 7   Lot Shape        2637 non-null   object 
 8   Land Contour     2637 non-null   object 
 9   Utilities        2637 non-null   object 
 10  Lot Config       2637 non-null   object 
 11  Land Slope       2637 non-null   object 
 12  Neighborhood     2637 non-null   object 
 13  Condition 1      2637 non-null   object 
 14  Condition 2      2637 non-null   object 
 15  Bldg Type        2637 non-null   object 
 16  House Style      2637 non-null   object 
 17  Overall Qual  

In [37]:
float_df = df.select_dtypes(include='float64')
#float_df['SalePrice'] = df['SalePrice']
#Making a copy of our dataframe so we don't modify the original, only looking at float data currently, but I included our target which is the sale price

In [38]:
int_df = df.select_dtypes(include='int64')
int_df

,PID,MS SubClass,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,1st Flr SF,2nd Flr SF,Low Qual Fin SF,...,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,SalePrice
0,905101070,20,14299,4,3,1964,1964,1005,0,0,...,0,0,0,0,0,0,0,7,2007,115400
1,905101330,90,10791,4,5,1967,1967,1296,0,0,...,0,0,0,0,0,0,500,10,2006,90000
2,903454090,50,9000,6,6,1937,1950,780,595,0,...,0,162,0,0,126,0,0,12,2007,141000
3,533244030,60,7379,8,5,2000,2000,975,873,0,...,280,184,0,0,0,0,0,4,2010,254000
4,909252020,70,7200,7,9,1936,2007,575,560,0,...,256,0,0,0,0,0,0,4,2009,155000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2632,903231070,50,6240,6,6,1934,1950,816,0,360,...,112,0,0,0,0,0,400,9,2006,114500
2633,906201021,80,10778,7,6,1990,1991,1061,0,0,...,114,36,0,0,0,0,0,7,2009,162000
2634,533253070,120,3782,8,5,1981,1981,1226,0,0,...,133,78,0,0,0,0,0,9,2009,211500
2635,527376100,20,10140,6,5,1974,1974,1350,0,0,...,0,0,0,0,0,0,0,8,2009,165000


In [39]:
num_df = float_df.join(int_df)
num_df


,Lot Frontage,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Bsmt Full Bath,Bsmt Half Bath,Garage Yr Blt,Garage Cars,...,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,SalePrice
0,62.0,0.0,513.0,144.0,348.0,1005.0,1.0,0.0,1964.0,2.0,...,0,0,0,0,0,0,0,7,2007,115400
1,72.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1967.0,2.0,...,0,0,0,0,0,0,500,10,2006,90000
2,50.0,0.0,0.0,0.0,780.0,780.0,0.0,0.0,1979.0,1.0,...,0,162,0,0,126,0,0,12,2007,141000
3,68.0,0.0,484.0,0.0,491.0,975.0,1.0,0.0,2000.0,2.0,...,280,184,0,0,0,0,0,4,2010,254000
4,60.0,0.0,350.0,210.0,0.0,560.0,1.0,0.0,1971.0,2.0,...,256,0,0,0,0,0,0,4,2009,155000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2632,52.0,0.0,0.0,0.0,816.0,816.0,0.0,0.0,1985.0,2.0,...,112,0,0,0,0,0,400,9,2006,114500
2633,74.0,0.0,460.0,286.0,308.0,1054.0,1.0,0.0,1990.0,2.0,...,114,36,0,0,0,0,0,7,2009,162000
2634,61.0,0.0,960.0,0.0,266.0,1226.0,1.0,0.0,1981.0,2.0,...,133,78,0,0,0,0,0,9,2009,211500
2635,78.0,174.0,0.0,0.0,1064.0,1064.0,0.0,0.0,1974.0,2.0,...,0,0,0,0,0,0,0,8,2009,165000


In [40]:
corr = num_df.corr()
corr.style.background_gradient(cmap='coolwarm')

,Lot Frontage,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Bsmt Full Bath,Bsmt Half Bath,Garage Yr Blt,Garage Cars,Garage Area,PID,MS SubClass,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,TotRms AbvGrd,Fireplaces,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,SalePrice
Lot Frontage,1.000000,0.223228,0.212286,0.034431,0.121012,0.355094,0.109700,-0.045547,0.088540,0.301058,0.351721,-0.106530,-0.421410,0.589199,0.208400,-0.068218,0.129555,0.103348,0.460991,0.020420,-0.004383,0.379570,0.179684,0.041521,0.232933,0.003988,0.345269,0.240883,0.119160,0.164550,-0.010544,0.020147,0.069019,0.160367,0.013259,0.023390,-0.005627,0.357142
Mas Vnr Area,0.223228,1.000000,0.285887,-0.013610,0.087586,0.382242,0.140490,-0.000873,0.249970,0.354833,0.366407,-0.243582,0.000571,0.137314,0.422037,-0.132828,0.310158,0.194648,0.378093,0.125080,-0.056658,0.393753,0.258576,0.190104,0.078022,-0.054249,0.269584,0.273639,0.152719,0.134065,-0.106207,0.025331,0.056370,0.011071,-0.025572,0.001584,-0.016366,0.506402
BsmtFin SF 1,0.212286,0.285887,1.000000,-0.060983,-0.489889,0.524524,0.643453,0.069977,0.181010,0.244749,0.292398,-0.110868,-0.051380,0.188551,0.267938,-0.039678,0.270672,0.141937,0.436187,-0.163901,-0.070001,0.189102,0.064414,-0.012565,-0.118179,-0.083208,0.028415,0.291439,0.217693,0.100365,-0.101918,0.070826,0.081536,0.093960,0.016907,-0.001286,0.027120,0.427809
BsmtFin SF 2,0.034431,-0.013610,-0.060983,1.000000,-0.238815,0.095250,0.171051,0.094488,-0.067357,-0.010261,0.006097,-0.000439,-0.072879,0.055292,-0.039208,0.043001,-0.026485,-0.056571,0.089630,-0.100145,-0.005277,-0.017734,-0.083961,-0.032809,-0.041190,-0.036407,-0.053746,0.066133,0.103607,-0.004739,0.038292,-0.022012,0.075109,0.045609,-0.001558,-0.006285,0.007498,0.009329
Bsmt Unf SF,0.121012,0.087586,-0.489889,-0.238815,1.000000,0.406733,-0.410715,-0.107476,0.172337,0.173811,0.158472,-0.084001,-0.137028,0.026916,0.270992,-0.145399,0.130946,0.167613,0.285954,-0.006209,0.041675,0.219396,0.271097,-0.037081,0.173493,0.047951,0.238319,-0.004720,-0.060960,0.119040,0.001504,-0.019425,-0.051316,-0.036474,-0.029649,0.021913,-0.033127,0.174628
Total Bsmt SF,0.355094,0.382242,0.524524,0.095250,0.406733,1.000000,0.325365,0.002069,0.340302,0.428430,0.469286,-0.201462,-0.222137,0.246816,0.539730,-0.171863,0.405522,0.295965,0.782696,-0.218010,-0.033055,0.413480,0.308910,-0.063966,0.035938,-0.053002,0.250098,0.326708,0.207495,0.223966,-0.089740,0.045551,0.063376,0.079599,-0.013036,0.018377,-0.002243,0.628669
Bsmt Full Bath,0.109700,0.140490,0.643453,0.171051,-0.410715,0.325365,1.000000,-0.147495,0.144726,0.154359,0.177041,-0.044396,0.012702,0.112005,0.161645,-0.034883,0.205280,0.125702,0.258250,-0.164186,-0.047282,0.051809,-0.030447,-0.041253,-0.164499,-0.018892,-0.054431,0.169862,0.190250,0.075758,-0.074047,0.043742,0.044927,0.047004,-0.013106,-0.006736,0.049217,0.277328
Bsmt Half Bath,-0.045547,-0.000873,0.069977,0.094488,-0.107476,0.002069,-0.147495,1.000000,-0.050460,-0.027729,-0.021823,0.010623,0.002473,0.027092,-0.048501,0.084600,-0.024301,-0.049207,-0.002806,-0.066461,-0.024364,-0.062032,-0.056694,-0.058726,0.015664,-0.051021,-0.052628,0.034449,0.056424,-0.045923,-0.020791,0.039823,0.043518,0.057100,-0.003968,0.023876,-0.019712,-0.042482
Garage Yr Blt,0.088540,0.249970,0.181010,-0.067357,0.172337,0.340302,0.144726,-0.050460,1.000000,0.576707,0.551783,-0.259524,0.092351,-0.004760,0.569697,-0.327586,0.831714,0.651249,0.244596,0.093045,-0.036570,0.268822,0.488952,0.224449,-0.048710,-0.099452,0.159419,0.082731,0.208013,0.221893,-0.303064,0.023574,-0.076077,-0.004225,-0.025041,0.024705,-0.006543,0.529771
Garage Cars,0.301058,0.354833,0.244749,-0.010261,0.173811,0.428430,0.154359,-0.027729,0.576707,1.000000,0.892040,-0.247019,-0.047860,0.201440,0.593861,-0.167497,0.529792,0.426257,0.427381,0.183537,-0.066671,0.482636,

In [41]:
corr = num_df.corrwith(df["SalePrice"])
# Our correlation matrix of our integer data shows some clear important features to consider, Overall quality, Gr Liv Area, and Garage Cars are the most correlated with our target, "SalePrice". But we also have to factor in all the categorical columns, which we will do further down.
corr.sort_values(ascending=False)

SalePrice          1.000000
Overall Qual       0.805016
Gr Liv Area        0.705482
Garage Cars        0.647223
Garage Area        0.633900
Total Bsmt SF      0.628669
1st Flr SF         0.618247
Year Built         0.562777
Full Bath          0.542386
Year Remod/Add     0.539932
Garage Yr Blt      0.529771
Mas Vnr Area       0.506402
TotRms AbvGrd      0.493853
Fireplaces         0.468886
BsmtFin SF 1       0.427809
Lot Frontage       0.357142
Wood Deck SF       0.322060
Open Porch SF      0.310361
Half Bath          0.287063
Bsmt Full Bath     0.277328
Lot Area           0.268516
2nd Flr SF         0.268059
Bsmt Unf SF        0.174628
Bedroom AbvGr      0.131789
Screen Porch       0.091214
Pool Area          0.072427
Mo Sold            0.052800
3Ssn Porch         0.040625
BsmtFin SF 2       0.009329
Misc Val          -0.019955
Yr Sold           -0.030053
Low Qual Fin SF   -0.040838
Bsmt Half Bath    -0.042482
MS SubClass       -0.078857
Overall Cond      -0.095205
Kitchen AbvGr     -0

In [42]:
# First we will break up our data into training and testing sets
#X = float_df[['Garage Cars', 'Garage Area', 'Total Bsmt SF', 'Mas Vnr Area']]
#X2 = sm.add_constant(X) # add constant so that intercept is estimated
#print(X2.head())

#y = float_df['SalePrice']
#X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size = 0.2, random_state=4)

In [43]:
sale_price_ds = df.iloc[:, 80]
sale_price_ds #Sale Price

0       115400
1        90000
2       141000
3       254000
4       155000
         ...  
2632    114500
2633    162000
2634    211500
2635    165000
2636    225000
Name: SalePrice, Length: 2637, dtype: int64

In [44]:
df.isna().sum()

PID                 0
MS SubClass         0
MS Zoning           0
Lot Frontage      449
Lot Area            0
                 ... 
Mo Sold             0
Yr Sold             0
Sale Type           0
Sale Condition      0
SalePrice           0
Length: 81, dtype: int64

In [45]:
cat_df = df.select_dtypes(include=['object'])
cat_df

,MS Zoning,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,...,Garage Type,Garage Finish,Garage Qual,Garage Cond,Paved Drive,Pool QC,Fence,Misc Feature,Sale Type,Sale Condition
0,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Sawyer,Feedr,...,Detchd,Unf,TA,TA,N,NaN,MnPrv,NaN,WD,Normal
1,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Sawyer,Norm,...,CarPort,Unf,TA,TA,Y,NaN,NaN,Shed,WD,Normal
2,RM,Pave,NaN,Reg,Bnk,AllPub,Inside,Gtl,IDOTRR,Norm,...,Detchd,Unf,TA,TA,P,NaN,NaN,NaN,WD,Normal
3,FV,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Somerst,Norm,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
4,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,SWISU,Feedr,...,Detchd,RFn,TA,TA,Y,NaN,MnPrv,NaN,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2632,RM,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,BrkSide,Norm,...,Detchd,Unf,TA,TA,Y,NaN,MnPrv,Shed,WD,Normal
2633,RL,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,SawyerW,Feedr,...,Attchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Normal
2634,RL,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,Greens,Norm,...,Attchd,Fin,TA,TA,Y,NaN,NaN,NaN,WD,Normal
2635,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,...,Attchd,RFn,TA,TA,Y,NaN,MnPrv,NaN,WD,Normal


In [46]:
cat_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2637 entries, 0 to 2636
Data columns (total 43 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   MS Zoning       2637 non-null   object
 1   Street          2637 non-null   object
 2   Alley           180 non-null    object
 3   Lot Shape       2637 non-null   object
 4   Land Contour    2637 non-null   object
 5   Utilities       2637 non-null   object
 6   Lot Config      2637 non-null   object
 7   Land Slope      2637 non-null   object
 8   Neighborhood    2637 non-null   object
 9   Condition 1     2637 non-null   object
 10  Condition 2     2637 non-null   object
 11  Bldg Type       2637 non-null   object
 12  House Style     2637 non-null   object
 13  Roof Style      2637 non-null   object
 14  Roof Matl       2637 non-null   object
 15  Exterior 1st    2637 non-null   object
 16  Exterior 2nd    2637 non-null   object
 17  Mas Vnr Type    1030 non-null   object
 18  Exter Qu

In [47]:
cat_df.drop("Misc Feature", axis = 1, inplace=True)
cat_df.drop("Pool QC", axis = 1, inplace=True)
cat_df.drop("Fence", axis = 1, inplace=True)
cat_df.drop("Mas Vnr Type", axis = 1, inplace=True)
cat_df.drop("Alley", axis = 1, inplace=True)
cat_df.drop("Fireplace Qu", axis = 1, inplace=True)


In [48]:
cat_df.info()
# Since we have so many features to look at, dropping the features that were very incomplete is the best route to go. 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2637 entries, 0 to 2636
Data columns (total 37 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   MS Zoning       2637 non-null   object
 1   Street          2637 non-null   object
 2   Lot Shape       2637 non-null   object
 3   Land Contour    2637 non-null   object
 4   Utilities       2637 non-null   object
 5   Lot Config      2637 non-null   object
 6   Land Slope      2637 non-null   object
 7   Neighborhood    2637 non-null   object
 8   Condition 1     2637 non-null   object
 9   Condition 2     2637 non-null   object
 10  Bldg Type       2637 non-null   object
 11  House Style     2637 non-null   object
 12  Roof Style      2637 non-null   object
 13  Roof Matl       2637 non-null   object
 14  Exterior 1st    2637 non-null   object
 15  Exterior 2nd    2637 non-null   object
 16  Exter Qual      2637 non-null   object
 17  Exter Cond      2637 non-null   object
 18  Foundati

In [68]:
for col in cat_df:
    print(df[col].value_counts()/df[col].notna().sum())
    
    # Here we are looking at the distribution in our categorical data sets. Since these are nominal data sets, we can hot encode our columns

MS Zoning
RL         0.774744
RM         0.158893
FV         0.046644
C (all)    0.009480
RH         0.008722
I (all)    0.000758
A (agr)    0.000758
Name: count, dtype: float64
Street
Pave    0.995449
Grvl    0.004551
Name: count, dtype: float64
Alley
Grvl    0.616667
Pave    0.383333
Name: count, dtype: float64
Lot Shape
Reg    0.630262
IR1    0.339401
IR2    0.024649
IR3    0.005688
Name: count, dtype: float64
Land Contour
Lvl    0.896852
HLS    0.042093
Bnk    0.040197
Low    0.020857
Name: count, dtype: float64
Utilities
AllPub    0.998862
NoSewr    0.000758
NoSeWa    0.000379
Name: count, dtype: float64
Lot Config
Inside     0.729617
Corner     0.174061
CulDSac    0.060675
FR2        0.030717
FR3        0.004930
Name: count, dtype: float64
Land Slope
Gtl    0.952218
Mod    0.042473
Sev    0.005309
Name: count, dtype: float64
Neighborhood
NAmes      0.150550
CollgCr    0.091013
OldTown    0.082670
Edwards    0.063709
Somerst    0.061054
NridgHt    0.056883
Gilbert    0.056504
Sawy

In [51]:
null_rows = cat_df[cat_df.isna().any(axis=1)]

print(null_rows)
null_drop_df = cat_df.drop(null_rows, axis =1)

     MS Zoning Street Lot Shape Land Contour Utilities Lot Config Land Slope  \
1           RL   Pave       Reg          Lvl    AllPub     Inside        Gtl   
42          RL   Pave       Reg          Lvl    AllPub     Corner        Gtl   
52          RL   Pave       Reg          Lvl    AllPub     Corner        Gtl   
71          RM   Pave       Reg          Lvl    AllPub     Inside        Gtl   
82          RM   Pave       Reg          Lvl    AllPub     Inside        Gtl   
...        ...    ...       ...          ...       ...        ...        ...   
2567        RM   Pave       Reg          Lvl    AllPub     Corner        Gtl   
2603        RL   Pave       Reg          Lvl    AllPub     Inside        Gtl   
2604        RM   Pave       Reg          Lvl    AllPub     Inside        Gtl   
2608        RL   Pave       Reg          Lvl    AllPub     Corner        Gtl   
2624        RL   Pave       Reg          Lvl    AllPub     Inside        Gtl   

     Neighborhood Condition 1 Condition

In [52]:
null_drop_df

""
0
1
2
3
4
...
2632
2633
2634
2635


In [66]:
mode = cat_df['Bsmt Qual'].value_counts().index[0]

# Fill missing values with the mode
cat_df['Bsmt Qual'].fillna(mode, inplace=True)

/var/folders/nl/cvxy58ks2mg_jyg002l659xc0000gn/T/ipykernel_17823/3078647145.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  cat_df['Bsmt Qual'].fillna(mode, inplace=True)


In [ ]:
 19  Bsmt Qual       2564 
 20  Bsmt Cond       2564 
 21  Bsmt Exposure   2561 
 22  BsmtFin Type 1  2564 
 23  BsmtFin Type 2  2563 
 30  Garage Type     2492 
 31  Garage Finish   2490 
 32  Garage Qual     2490 
 33  Garage Cond     2490 

In [53]:
cat_df2 = cat_df.copy()


In [63]:
imp = SimpleImputer(missing_values = np.nan, strategy = 'most_frequent')
simp_impute_cat_df2 = imp.fit_transform(cat_df2)
pd.DataFrame( simp_impute_cat_df2, columns = simp_impute_cat_df2.Index(range(2637)))

AttributeError: 'numpy.ndarray' object has no attribute 'Index'

In [56]:
imp = IterativeImputer(max_iter=10, random_state = 4)
my_df_iter_impute = imp.fit_transform(my_)
pd.DataFrame( my_df_iter_impute, columns = my_df.columns )

NameError: name 'my_df' is not defined

In [27]:
# Define the columns
A = ['Alley']
B = ['Foundation'] #'Has_Alley'] # Example of multiple columns in B
# Count values for A and B
counts_A = cat_df[A].value_counts(dropna=True).reset_index()
counts_B = cat_df[B].value_counts(dropna=True).reset_index()

# Merge the dataframes on multiple columns in B
merged_df = pd.merge(
    counts_A,
    counts_B,
    on=B,
    suffixes=('_A', '_B'))
#)

# Calculate marginal probabilities
merged_df['marg_prob'] = merged_df['count_A'] / merged_df['count_B'] * 100

# Display the resulting dataframe
merged_df

KeyError: "None of [Index(['Alley'], dtype='object')] are in the [columns]"

In [16]:
(df["Street"] == 'Pave').sum(), (df["Foundation"] == 'PConc').sum()


(np.int64(2625), np.int64(1185))

In [18]:
#(df["Street"] == 'Pave') & (df["Foundation"] == 'PConc').sum()/len(df)
# This is a joint probability

In [19]:
((df["Foundation"] == 'PConc') & (df["Street"] == 'Pave')).sum()/len(df)

np.float64(0.44899507015547974)

In [20]:
df[['Street', 'Foundation']].value_counts() / df[['Street', 'Foundation']].value_counts().sum()

Street  Foundation
Pave    PConc         0.448995
        CBlock        0.419795
        BrkTil        0.105044
        Slab          0.016686
        Stone         0.003792
Grvl    CBlock        0.003413
Pave    Wood          0.001138
Grvl    BrkTil        0.000379
        PConc         0.000379
        Stone         0.000379
Name: count, dtype: float64

In [19]:
# Assuming df is your DataFrame with 43 categorical columns
cols = float_df.columns

joint_probabilities = {}

# Loop through each pair of columns
for i in range(len(cols)):
    for j in range(i + 1, len(cols)):
        col1, col2 = cols[i], cols[j]
        # Create cross-tabulation (contingency table)
        crosstab = pd.crosstab(df[col1], df[col2])
        # Normalize by total number of rows to get joint probability
        joint_prob = crosstab / len(df)
        # Store in dictionary
        joint_probabilities[(col1, col2)] = joint_prob

# joint_probabilities dictionary now holds joint probabilities for each pair of columns

In [21]:
pairs = ['Street', 'Heating']
df[pairs].value_counts()/df[pairs].value_counts().sum()*100

# for all of our categorical pairs we have 43 total columns, and so we have ((43*43)-43)/2. subtract 43 because we don't need to compare against itself and divide by 2 since there are pairs

Street  Heating
Pave    GasA       98.028062
        GasW        0.872203
Grvl    GasA        0.455063
Pave    Grav        0.341297
        Wall        0.189609
        OthW        0.075844
        Floor       0.037922
Name: count, dtype: float64

In [18]:
# Fit a linear model using statsmodels
myfit = sm.OLS(y_train, X_train).fit()
myfit.summary()

NameError: name 'y_train' is not defined

In [41]:
housing_df = df


In [43]:
categorical_cols = housing_df.select_dtypes(include = 'object')

categorical_cols['Neighborhood'].value_counts()

categorical_cols.transpose()

for col in categorical_cols:
    print(f"Value counts for {col}:")
    total = len(housing_df)
    print(categorical_cols[col].value_counts(dropna = False)/ total * 100)
    print("\n")

pairs = ['Street', 'Heating'] #and... how many pairs are there in a symmetrical matrix? (don't include the diagonal, itself with itself) 43x43 - 43(take out diagonals)/2 (because only need one triangle of the matrix): 43x21 = 903 possible pairs
categorical_cols[pairs].value_counts()/categorical_cols[pairs].value_counts().sum()*100

A = 'Street'
B = 'Heating'


pairs = [A, B] #and... how many pairs are there in a symmetrical matrix? (don't include the diagonal, itself with itself) 43x43 - 43(take out diagonals)/2 (because only need one triangle of the matrix): 43x21 = 903 possible pairs

categorical_cols[pairs].value_counts()/(2585 + 12)
categorical_cols[pairs].value_counts()/(categorical_cols[B] == 'GasA').sum()
categorical_cols[pairs].value_counts() , categorical_cols[B].value_counts()
(categorical_cols[pairs].value_counts()).reset_index() ,
categorical_cols[B].value_counts().reset_index()

((categorical_cols[pairs].value_counts()).reset_index() ,
categorical_cols[B].value_counts().reset_index())

merged_df = pd.merge((categorical_cols[pairs].value_counts()).reset_index() ,
categorical_cols[B].value_counts().reset_index(), on = B)
merged_df

merged_df['marg_prob'] = merged_df['count_x']/merged_df['count_y'] * 100
merged_df

B =['Street']
A = ['Alley'] #shout out to the pc haters #and... how many pairs are there in a symmetrical matrix? (don't include the diagonal, itself with itself) 43x43 - 43(take out diagonals)/2 (because only need one triangle of the matrix): 43x21 = 903 possible pairs
merged_df = pd.merge(
    categorical_cols[A + B].value_counts(dropna = True).reset_index() ,
    categorical_cols[B].value_counts(dropna = True).reset_index(),
    on = B
  )
merged_df['marg_prob'] = merged_df['count_x']/merged_df['count_y'] * 100
merged_df

categorical_cols.head()

# Define the columns
B = ['Street', 'Utilities']  # Example of multiple columns in B
A = ['Alley']
A = A + B


# Count values for A and B
counts_A = categorical_cols[A].value_counts(dropna=False).reset_index()
counts_B = categorical_cols[B].value_counts(dropna=False).reset_index()
counts_A, counts_B

# Merge the dataframes on multiple columns in B
merged_df = pd.merge(
    counts_A,
    counts_B,
    on=B,
    suffixes=('_A', '_B')
)
merged_df


# Calculate marginal probabilities
merged_df['marg_prob'] = merged_df['count_A'] / merged_df['count_B'] * 100

# Display the resulting dataframe
merged_df

categorical_cols['Has_Alley'] = categorical_cols['Alley'].notna()
# Define the columns
A = ['Alley']
B = ['Street', 'Utilities', 'Has_Alley'] # Example of multiple columns in B

# Count values for A and B
counts_A = categorical_cols[A + B].value_counts(dropna=False).reset_index()
counts_B = categorical_cols[B].value_counts(dropna=False).reset_index()

# Merge the dataframes on multiple columns in B
merged_df = pd.merge(
    counts_A,
    counts_B,
    on=B,
    suffixes=('_A', '_B')
)


# Calculate marginal probabilities
merged_df['marg_prob'] = merged_df['count_A'] / merged_df['count_B'] * 100

# Display the resulting dataframe
merged_df



## Exploratory Data Analysis

Value counts for MS Zoning:
MS Zoning
RL         77.474403
RM         15.889268
FV          4.664391
C (all)     0.948047
RH          0.872203
I (all)     0.075844
A (agr)     0.075844
Name: count, dtype: float64


Value counts for Street:
Street
Pave    99.544937
Grvl     0.455063
Name: count, dtype: float64


Value counts for Alley:
Alley
NaN     93.174061
Grvl     4.209329
Pave     2.616610
Name: count, dtype: float64


Value counts for Lot Shape:
Lot Shape
Reg    63.026166
IR1    33.940083
IR2     2.464922
IR3     0.568828
Name: count, dtype: float64


Value counts for Land Contour:
Land Contour
Lvl    89.685248
HLS     4.209329
Bnk     4.019719
Low     2.085703
Name: count, dtype: float64


Value counts for Utilities:
Utilities
AllPub    99.886234
NoSewr     0.075844
NoSeWa     0.037922
Name: count, dtype: float64


Value counts for Lot Config:
Lot Config
Inside     72.961699
Corner     17.406143
CulDSac     6.067501
FR2         3.071672
FR3         0.492984
Name: count, dtype: fl

,Alley,Street,Utilities,Has_Alley,count_A,count_B,marg_prob
0,NaN,Pave,AllPub,False,2443,2443,100.000000
1,Grvl,Pave,AllPub,True,111,180,61.666667
2,Pave,Pave,AllPub,True,69,180,38.333333
3,NaN,Grvl,AllPub,False,11,11,100.000000
4,NaN,Grvl,NoSewr,False,1,1,100.000000
5,NaN,Pave,NoSeWa,False,1,1,100.000000
6,NaN,Pave,NoSewr,False,1,1,100.000000
